## Network Training

모델의 훈련에는 다양한 요소가 들어있습니다. 이전에 알아둔 모델의 생성 이후 우리는 어떤 고민을 해야하고 어떤 해결방법을 택할 수 있는지 알아보겠습니다

### trainable

훈련 가능하다는 것은 뭐일까? 

아래의 예시를 살펴보자

In [3]:
from zipfile import ZipFile
import io
with ZipFile("../Data/고객 대출등급 분류 해커톤.zip","r") as f:
    data=io.BytesIO(f.read("고객 대출등급 분류 해커톤/train.csv"))

from pandas import read_csv
data=read_csv(data)

import keras

In [8]:
data[["대출금액"]].head(3)

,대출금액
0,12480000
1,14400000
2,12000000


이전 예시에서 우리는 위의 특성에 대해 `Standard Scale`을 적용했습니다.(즉 정규화)

이를 다음과 같이 진행할 수도 있습니다

In [54]:
import tensorflow as tf
variance=tf.constant(data[["대출금액"]].values)

In [91]:
# 정규화 층 설정
standard_layers=keras.layers.Normalization()
standard_layers.adapt(variance)

In [93]:
# 은닉층 예시
hidden_layer=keras.layers.Dense(1,activation="relu")
hidden_layer.build(input_shape=(1,))

아래의 정규화층과 은닉층의 학습가능한 변수를 확인해보죠

아래와 같이 정규화 층은 어떠한 연산곱을 실행하지 않기때문에(다만 `adapt`메소드를 통해 분산과 평균값을 가지고 있습니다)

은닉층(Dense)층과 다르게 훈련가능한 변수가 없습니다.

In [96]:
standard_layers.trainable_variables

[]

In [95]:
hidden_layer.trainable_variables

[<Variable path=dense_6/kernel, shape=(1, 1), dtype=float32, value=[[1.303249]]>,
 <Variable path=dense_6/bias, shape=(1,), dtype=float32, value=[0.]>]

In [90]:
standard_layers(variance)[:5]

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[-0.56384814],
       [-0.37796414],
       [-0.61031914],
       [-0.37796414],
       [-0.02943163]], dtype=float32)>

In [99]:
hidden_layer(standard_layers(variance)[:5])

<tf.Tensor: shape=(5, 1), dtype=float32, numpy=
array([[0.],
       [0.],
       [0.],
       [0.],
       [0.]], dtype=float32)>

여기서 요점은 trainable variance 가 훈련 중 수정될 주요 변수라는 점이다.

### 훈련 방법

모델의 훈련 방법은 자동후진미분을 통해 이루어집니다.

또한 전통적인 방법으로 경사하강법을 적용하는데, 이에 대해 알아보겠습니다.

#### Loss(손실률)

역전파를 통한 파라미터를 수정하는 과정에서 우선적으로 `loss`(손실량)를 결정해야합니다.

여기서 당연하겠지만, 우리는 손실량을 __최소__ 로 하는게 목적입니다.

예를 들어 loss를 `accuracy`로 결정하는 것은 목적에 부합하지않고, error rate=1-accuracy 로 손실을 설정하는게 

목적에 부합한 loss 설정방법입니다.

Tensorflow(Keras)에서는 Loss 를 살펴보죠

In [102]:
# 이진분류의 엔트로피
keras.losses.BinaryCrossentropy

keras.src.losses.losses.BinaryCrossentropy

In [ ]:
# SVM 의 최대 마진 손실함수
keras.losses.Hinge

In [ ]:
# 이상치 영향을 완화하기 위한 손실함수
keras.losses.Huber

기본적으로 우리가 자주 접하는 MSE, MAE 등 다양한 손실함수를 지원하므로 한번씩 살펴보도록 하자.

#### Gradient

위의 손실함수가 결정되었다면 우리는 역전파를 통해 각 노드의 gradient 를 계산할 수 있습니다.

이는 `Tensorflow.GradientTape`를 활용하면 쉽게 살펴볼 수 있습니다.

간단한 모델을 만들어 살펴보죠.

1. 연속형 특성을 사용하죠.
2. 은닉층은 2층만 사용하겠습니다.

In [107]:
import numpy as np
use_feature=data.select_dtypes(np.number)
use_label=data.대출등급

In [110]:
label_list=data.대출등급.unique()
label_list.sort()

In [118]:
# 숫자로 변환
use_label=use_label.map(lambda x:np.argmax(label_list==x))

In [120]:
use_feature.values

array([[1.24800e+07, 7.20000e+07, 1.89000e+01, ..., 0.00000e+00,
        0.00000e+00, 0.00000e+00],
       [1.44000e+07, 1.30800e+08, 2.23300e+01, ..., 2.34060e+05,
        0.00000e+00, 0.00000e+00],
       [1.20000e+07, 9.60000e+07, 8.60000e+00, ..., 1.51944e+05,
        0.00000e+00, 0.00000e+00],
       ...,
       [1.44000e+07, 8.40000e+07, 1.12400e+01, ..., 2.41236e+05,
        0.00000e+00, 0.00000e+00],
       [1.56000e+07, 6.63300e+07, 1.73000e+01, ..., 8.18076e+05,
        0.00000e+00, 0.00000e+00],
       [8.64000e+06, 5.04000e+07, 1.18000e+01, ..., 2.74956e+05,
        0.00000e+00, 0.00000e+00]])

In [208]:
standard_layers=keras.layers.Normalization()
standard_layers.adapt(use_feature.values)

Simple_model=keras.Sequential(
 [keras.layers.Input(shape=use_feature.shape[-1:]),
  standard_layers,
  keras.layers.Dense(9,activation="relu"),
  keras.layers.Dense(9,activation="relu"),
  keras.layers.Dense(7,activation="sigmoid")]   
)

위의 간단한 모델에서 훈련 가능한 변수를 살펴보자.

In [209]:
# 마지막 output layer 층
Simple_model.trainable_weights[-2:]

[<Variable path=sequential_4/dense_21/kernel, shape=(9, 7), dtype=float32, value=[[-0.11383507 -0.00220287  0.06821764  0.5739718  -0.5091083   0.35995752
   -0.30687857]
  [ 0.4870692   0.2801087   0.06435692  0.458798    0.11078292  0.36766064
    0.03970134]
  [ 0.3055752  -0.51906633 -0.49382025 -0.41563368  0.26066095 -0.19154072
   -0.50225806]
  [ 0.3222494   0.5239642  -0.4780228  -0.5641748   0.56159717  0.03943264
   -0.4706429 ]
  [ 0.22212398 -0.17264467  0.19317448  0.5103157   0.32983398  0.40341145
   -0.2512633 ]
  [-0.44533098 -0.27005273 -0.5697107   0.52962214  0.43648583  0.4109463
    0.25538373]
  [-0.17670673  0.50366515  0.28261262 -0.19724512  0.28348935  0.15907198
   -0.59133196]
  [ 0.3777575  -0.16513804 -0.29112315  0.52439624 -0.33534116  0.15115249
    0.57314974]
  [ 0.43179315 -0.15831149 -0.41772968 -0.08764207  0.12076485 -0.53711
   -0.19404975]]>,
 <Variable path=sequential_4/dense_21/bias, shape=(7,), dtype=float32, value=[0. 0. 0. 0. 0. 0. 0.]>]

loss 선정 및 간단한 확인

In [210]:
use_loss=keras.losses.SparseCategoricalCrossentropy()

아래는 첫번째 데이터의 예측값을 나타냅니다.
(각 라벨당의 확률을 나타냄과 같은 효과입니다.)

In [223]:
pred_y=Simple_model(use_feature.values[:1,:])
print(pred_y)

print("예측된 y:",label_list[tf.argmax(pred_y[0])])
print("실제의 y:",label_list[use_label[0]])

tf.Tensor(
[[0.4827957  0.3738754  0.38036695 0.7468218  0.70034134 0.6690052
  0.47923845]], shape=(1, 7), dtype=float32)
예측된 y: D
실제의 y: C


In [225]:
# 손실값은 다음과 같이 계산됩니다.

use_loss(y_true=use_label[0],y_pred=pred_y[0])

<tf.Tensor: shape=(), dtype=float32, numpy=2.3101218>

gradient 를 계산해보죠

In [258]:
with tf.GradientTape() as tape:
    pred_y=Simple_model(use_feature.values[:1,:])
    loss=use_loss(use_label[0],pred_y[0])
grad=tape.gradient(loss,Simple_model.trainable_variables)

마지막 층(노드)에 해당하는 손실함수에 대한 gradient 는 다음과 같습니다

In [261]:
grad[-2:]

[<tf.Tensor: shape=(9, 7), dtype=float32, numpy=
 array([[ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.07311683,  0.06854559, -0.62633556,  0.05536497,  0.06145093,
          0.06483992,  0.07307728],
        [ 0.06593559,  0.06181332, -0.5648194 ,  0.04992725,  0.05541547,
          0.05847161,  0.06589993],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ],
        [ 0.02000165,  0.01875116, -0.17133872,  0.0151455 ,  0.0168103

#### Optimizer

드디어 위의 계산된 gradient를 적용할 차례입니다.

이를 `Tensorflow(keras)`에서 알아보자.

In [265]:
# 전통적인 경사하강법
sgd=keras.optimizers.SGD(learning_rate=0.01)

In [ ]:
sgd.apply_gradients(zip(grad,Simple_model.trainable_variables))

<Variable path=SGD/iteration, shape=(), dtype=int64, value=1>

마지막 층의 `bias`가 어떻게 갱신됐는지 보죠!

In [277]:
Simple_model.trainable_variables[-1:]

[<Variable path=sequential_4/dense_21/bias, shape=(7,), dtype=float32, value=[-0.00065155 -0.00061082  0.00558135 -0.00049336 -0.0005476  -0.0005778
  -0.0006512 ]>]

결과확인

In [279]:
pred_y=Simple_model(use_feature.values[:1,:])
print(pred_y)

tf.Tensor(
[[0.48339954 0.37522963 0.38878614 0.7444099  0.6976011  0.66783875
  0.47791767]], shape=(1, 7), dtype=float32)


기본적으로 네트워크 훈련의 아주 작은 스텝이 어떻게 이루어짐을 알아보았습니다.